# GEOG 5160 6160 Lab 07

## Data processing

Let's start by by importing the modules we'll need for the class, then we'll go get the data

In [1]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
## Set random seed for reproducibility
np.random.seed(1234)

### Pinus edulis data

Now, let's read in the known locations of *Pinus edulis* trees from the file *Pinus_edulis.csv*, and a set of pseudo-absences from the file *absence.csv*

In [2]:
presence = pd.read_csv("../datafiles/Pinus_edulis.csv")
presence.describe()

In [3]:
absence = pd.read_csv("../datafiles/absence.csv")
absence.describe()

We'll need to append these two datasets to have both presence and absence in the same DataFrame. 

- Append the coordinates
- Create a binary Series where 0 = absence and 1 = presence
- Convert to a Pandas DataFrame

In [4]:
lon = presence.longitude.append(absence.longitude).values
lat = presence.latitude.append(absence.latitude).values

pa = pd.Series([1, 0]).repeat(299).values

frame = { 'longitude': lon, 
         'latitude': lat,
         'pa': pa
        } 
pe = pd.DataFrame(frame)
pe.describe()

#### Environmental data

There are a large number of available data sources for environmental data that can be used in species distribution models. We'll take data from the [Worldclim project][wcID] (Hijmans et al. 2005), a collection of standardized climate data at a variety of spatial resolutions. The data contains monthly averages of temperature and precipitation and a set of bioclimatic variables, which represent aggregate climate variables assumed to be linked to species distributions. I've already downloaded the bioclimatic variables for you and clipped them to the region you're going to work in. These are available in two NetCDF files containing modern (*current_env.nc*) and future (*future_env.nc*) climates for the study area. 

NetCDF is a standard format for large, multidimensional gridded dataset, and is commonly used to store climate data. Python has several libraries that will read these files. We'll use `xarray` here, as it comes with some simple functions to extract values and plot the data that work well with these data. The output of the `info` method gives you an overview of the dataset, including the dimensions, coordinates and the variables. Each z-level in this file stores one of the 19 bioclimate variables (the description of each of these is given in the appendix). 

In [5]:
import xarray as xr
curr_env = xr.open_dataset('../datafiles/current_env.nc')
curr_env.info
#curr_env['variable'].sel(lon=-107, lat=40)

`xarray` introduces two new data objects to Python, a DataArray (which contain the variables we are interested in) and a DataSet, which is a collection of DataArrays. The `current_env` DataSet contains two DataArrays (`crs` and `BIO`). The second of these contains the bioclimate variables that we want to work with. 

In [6]:
curr_env.BIO

You can plot any individual layer using the `plot()` method. Note the indexing used to identify the layer to be shown (`BIO[0]` = the first layer):

In [7]:
curr_env.BIO[0].plot()

As the region we are using for our study area includes part of the Pacific Ocean, we'll create a land/sea mask. This will be used later to mask out any predictions over the ocean.  This is done quite simple by creating an array with 2's over the ocean (the NaN values in the data) and a second with 1's over the land. Then these are combined into the final mask.

In [8]:
mask_ocean = 2 * np.ones((curr_env.dims['latitude'], 
                          curr_env.dims['longitude'])) * np.isnan(curr_env.BIO.isel(z=0))  
mask_land = 1 * np.ones((curr_env.dims['latitude'], 
                         curr_env.dims['longitude'])) * np.isfinite(curr_env.BIO.isel(z=0))  
mask_array = mask_ocean + mask_land
mask_array.plot()

We also have the same data for the future (2080 under a high emissions climate scenario), so let's load that as well. Once we have built our model, we can then predict the distribution of Pinus edulis under this changed climate. 

In [9]:
future_env = xr.open_dataset('../datafiles/future_env.nc')
future_env.info

Now we have the presence and absence points, we need to extract the environmental values for these points (these will be a features for machine learning). To do this we use the following steps:

- Create sets of coordinates for the presence/absence data in `xarray` format
- Use the `sel()` method to extract the associated climate values from the environmental grids for these coordinates
- Convert the extracted data to a Pandas DataFrame
- Concatenate the presence/absence data frame with the environmental variables

In [10]:
lons = xr.DataArray(pe.longitude, dims='x')
lats = xr.DataArray(pe.latitude, dims='x')

In [11]:
data = curr_env.BIO.sel(latitude = lats, longitude = lons, method = 'nearest')
data.values

In [12]:
var_names = ["bio"+str(i+1) for i in range(19)]
x = pd.DataFrame(data.values.transpose(),
                columns = var_names)

In [13]:
pe = pd.concat([pe, x], axis=1)
pe

Before moving on, we'll convert this new DataFrame to a GeoPandas dataframe, so we can make some quick maps of the presence/absence values and one of the associated environmental variables (`bio7`). 

In [14]:
import geopandas as gpd
pe_gpd = gpd.GeoDataFrame(pe, 
                          geometry=gpd.points_from_xy(pe.longitude, pe.latitude), 
                               crs=4326)

In [15]:
pe_gpd.plot(column="pa", figsize = (6.5, 6.5), categorical=True, 
            markersize = 100, legend=True, edgecolor="black")

In [16]:
pe_gpd.plot(column="bio7", figsize = (6.5, 6.5), 
            markersize = 100, legend=True, edgecolor="black")

## Tree methods

### Classification and regression trees

Classification and Regression Trees (CART) is a non-linear, non-parametric modeling approach that can be used with a wide variety of data. Regression trees are used with continuous outcome data, and classification trees with binary or categorical data, but the interface for these is the same in scikit-learn.

We'll build a classification model for the *Pinus edulis* data set. First, let's set up the training and testing set using all 19 of the bioclimatic variables:

In [17]:
var_names = ["bio"+str(i+1) for i in range(19)]

X = pe[var_names]
y = pe['pa']

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size = 0.8)

In [19]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

No prizes for guessing that the scikit-learn function for a classification decision tree is called `DecisionTreeClassifier()`. Let's initialize one of these, and train it on our dataset

In [20]:
from sklearn import tree
pe_tree = tree.DecisionTreeClassifier()
pe_tree = pe_tree.fit(X_train, y_train)

And plot the resulting tree:

In [21]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,20)) 
tree.plot_tree(pe_tree)
plt.show()

Again, the default settings results in a very overfit tree. To illustrate this, let's calculate the AUC on the testing set:

In [22]:
from sklearn import metrics
y_test_pred = pe_tree.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_test_pred[:,1])

### Tuning

Let's try to improve on this model by tuning it to find the best set of hyperparameters to limit overfitting. You can get the list of available hyperparameters for any scikit-learn model using `get_params()`

In [23]:
tree.DecisionTreeClassifier().get_params()

There are several here that we could try to tune, but we'll focus on two that are important in limiting tree size:

- `max_depth`: the maximum number of splits along any branch of the tree
- `min_samples_leaf`: the minimum number of samples in a child node for it to be considered in the splitting procedure

We'll use a scikit-learn function to help with the tuning called `GridSearchCV()`. As the name might imply, this will carry out a cross-validated search among a set of hyperparameter values to find the best value. Practically, this takes the dataset, splits it into training and testing, builds several models with different parameter values and predicts for the test set. The best values are then saved and returned. 

After importing the function, we need to define the parameter space that will be searched. This is set up as a Python dictionary, with the name of the hyperparameter as the key, and the value or set of values to be tested. Any parameters that are not specified will be held at their default values. 

In [24]:
from sklearn.model_selection import GridSearchCV
tree_param = {'max_depth':[4,5,6,7,8,9,10],
             'min_samples_leaf':[5,6,7,8,9,10]}

Having done this, we set up the cross-validation strategy using `GridSearchCV()`. We need to specify:

- The algorithm to be tuned
- The parameter space
- The performance metric to be used to select the parameter values
- The cross-validation strategy. Here we are using a simple 5-fold cross-validation, but it is possible to replace this with more complex strategies

In [25]:
pe_tree_tuned = GridSearchCV(tree.DecisionTreeClassifier(), tree_param, 
                             scoring='roc_auc', cv=5)

Now we use the standard scikit-learn `fit()` method to run the tuning

In [26]:
pe_tree_tuned.fit(X_train, y_train)

The full set of results are held in `pe_tree_tuned.cv_results_`, but we'll just show here the range of AUC scores found during tuning. These vary a little (from just under 0.8 to about 0.83). Tuning has a relatively small impact on decision trees, so this small range is not too surprising.

In [27]:
print(pe_tree_tuned.cv_results_['mean_test_score'])

We can also extract the `best_estimator`, i.e. the best tree that was built to see the parameters that were used, and the score it obtained. 

In [28]:
print(pe_tree_tuned.best_estimator_)
print(pe_tree_tuned.best_score_)

In [29]:
print('Best max_depth:', pe_tree_tuned.best_estimator_.get_params()['max_depth'])
print('Best min_samples_leaf:', pe_tree_tuned.best_estimator_.get_params()['min_samples_leaf'])

Finally, let's re-predict for our test set and see if we've improved on the original default model

In [30]:
y_test_pred = pe_tree_tuned.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_test_pred[:,1])

## Random forest

Next, we'll build a random forest for the Pinus data. scikit-learn's random forest functions are found in the `ensemble` sub-module. Let's import this and then set up, train and test a random forest classifier. 

In [31]:
from sklearn import ensemble

In [32]:
pe_rf = ensemble.RandomForestClassifier()
pe_rf.fit(X_train, y_train)
y_test_pred = pe_rf.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_test_pred[:,1])

Even with its default settings, this model shows a decent improvement over the decision tree. We'll now try to tune it. First, look at the available parameters:

In [33]:
ensemble.RandomForestClassifier().get_params()

Again, there are several hyperparameters that we could tune. We'll focus again on two: `n_estimators` (the number of trees in the forest) and `max_features` (the number of randomly selected features used for each split)

In [34]:
rf_param = {'n_estimators':[100, 200, 300, 400, 500],
             'max_features':[2,4,6]}

We now run our tuning grid search using these parameters and a random forest:

In [35]:
pe_rf_tuned = GridSearchCV(ensemble.RandomForestClassifier(), rf_param, 
                             scoring='roc_auc', cv=5)
pe_rf_tuned.fit(X_train, y_train)
print(pe_rf_tuned.cv_results_['mean_test_score'])

And look at the best fitting model:

In [36]:
print(pe_rf_tuned.best_estimator_)
print(pe_rf_tuned.best_score_)

Now let's see if tuning the model has improved over the default (I got a very slight improvement, but your mileage may vary):

In [37]:
y_test_pred = pe_rf_tuned.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_test_pred[:,1])

#### Variable importance

Next we'll plot the permutation-based variable importance for this model. As a reminder, variable importance is a measure of how much worse a model becomes when we scramble the values of one of the features. The model is used to predict the outcome for some test data (here the out-of-bag samples) twice: once with the original values of the feature and once with randomly shuffled values. If there is a large difference in the skill of the model, this feature is important in controlling the outcome. 

As the result of the grid-based tuning includes the best estimator, we'll use this to get the variable importance values as follows:

In [38]:
pe_rf_tuned.best_estimator_.feature_importances_

As this is a just an array, it is a little difficult to parse out any differences. Instead, we'll sort them and print them together with the feature names:

In [39]:
importances = pe_rf_tuned.best_estimator_.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], var_names[indices[f]], importances[indices[f]]))

And we can also plot the values as a bar plot:

In [40]:
# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(X.shape[1]), importances[indices],
        color="r", align="center")
plt.xticks(range(X.shape[1]), indices)
plt.xlim([-1, X.shape[1]])
plt.show()

We can look at the form of the relationship between the occurrence of the pine and this feature (and any other one) using a partial dependency plot. This shows changes in the outcome across the range of some feature (with all other features held constant). Here, we'll use the `PartialDependenceDisplay()` function from the the `inspection` submodule to produce the plot. As arguments, this requires the model, the DataFrame or array used to build the model, and the feature that you want to show. This will take an array of feature indices, allowing you to plot several dependency plots together. 

If you are using an earlier version of scikit-learn (pre 1.2), then the function is called `plot_partial_dependence()`


In [41]:
from sklearn.inspection import PartialDependenceDisplay
PartialDependenceDisplay.from_estimator(pe_rf_tuned.best_estimator_, X, [16,0])

The two features here are `bio17` (precipitation of driest quarter) and `bio1` (mean annual temperature. The first of these shows a clear threshold effect, with an abrupt drop in suitability as this drops below about 30. The second show more of an optimum between values of about 6 and 10 degrees.

### XGBoost

We will now build a boosted regression tree model for the Pinus data. In contrast to random forests that build a set of individual weak trees, boosted regression trees (BRTs) start with a single weak tree and iteratively improve on this. This is done by targeting the residuals from the previous set of models and trying to model that in the next tree. While this can make these methods very powerful, it is easy for them to overfit the data, and hyperparameter tuning becomes very important here. 

While scikit-learn has its own implementation of boosted regression, an alternative is through the xgboost library. This has a number of advantages with memory management and parallelization which can greatly speed up fitting these models, and we'll use this. It has a very similar interface to scikit-learn, so we can just reuse the data and tuning approaches from previous models. As before, we'll start by simply running it with the default settings:

In [42]:
from xgboost import XGBClassifier
pe_xgb = XGBClassifier()
pe_xgb.fit(X_train, y_train)
y_test_pred = pe_xgb.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_test_pred[:,1])

Now we'll try to tune it. First, get a list of available parameters

In [43]:
XGBClassifier().get_params()

Well, there's a lot. A thorough search strategy would probably include some of the following parameters:

In [44]:
# Define our search space for grid search
xgb_param = {
            'max_depth': [3, 4, 5],
            'learning_rate': [0.1, 0.2, 0.3],
            'n_estimators': [50, 100, 150],
            'gamma': [0, 0.1, 0.2],
            'min_child_weight': [0, 0.5, 1],
            'max_delta_step': [0],
            'subsample': [0.7, 0.8, 0.9, 1],
            'colsample_bytree': [0.6, 0.8, 1],
            'colsample_bylevel': [1],
            'reg_alpha': [0, 1e-2, 1, 1e1],
            'reg_lambda': [0, 1e-2, 1, 1e1],
            'base_score': [0.5]
            }

In the interest of keeping this relatively fast, we'll just test the following parameters, each with a fairly coarse grid. In practice, you'd want to do this more exhaustively. 

- `max_depth`: the number of splits in each tree
- `learning_rate`: the contribution of each tree to the overall model
- `n_estimators`: the total number of trees built

We also set the parameter `subsample` to 0.5 to only use a random selection of observations in building each tree. As this is a constant, it won't be varied during the grid search. Once this is setup, we'll run the usual cross-validated grid search. You might see a few warnings appear, you can safely ignore these.

In [45]:
xgb_param = {
    'max_depth': [1, 3, 5],
    'learning_rate': [0.1, 0.2, 0.3],
    'n_estimators': [100, 300, 500],
    'subsample': [0.5], 
    'eval_metric': ['logloss']
}

pe_xgb_tuned = GridSearchCV(XGBClassifier(), xgb_param, 
                             scoring='roc_auc', cv=5, verbose = 0)
pe_xgb_tuned.fit(X_train, y_train)

In [46]:
print(pe_xgb_tuned.best_estimator_)
print(pe_xgb_tuned.best_score_)

Now run the model to predict for our test samples. This does give a small but notable increase over the un-tuned model. 

In [47]:
y_test_pred = pe_xgb_tuned.predict_proba(X_test)
metrics.roc_auc_score(y_test, y_test_pred[:,1])

We can again extract the variable importance scores from the tuned model, which again shows `bio17` as being the most importance feature. 

In [48]:
importances = pe_xgb_tuned.best_estimator_.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d %s (%f)" % (f + 1, indices[f], var_names[indices[f]], importances[indices[f]]))

## Predictions

Now we tested and tuned our models, we can use them for prediction. For species distribution models, we generally want to predict the suitability for our species using a gridded dataset of the environmental variables used to build the model. We loaded this earlier in the lab as `curr_env`. As a reminder, this is an xarray `DataSet`. 


In [49]:
curr_env.info

Unfortunately, we can't use this directly for predictions. We'll need to convert it to a Pandas DataFrame. This requires a couple of steps. First, we use the `stack()` method to collapse the multiple levels into a 2D array, where the rows represent the variables and the columns represent individual grid locations:

In [50]:
bio_stack = curr_env.BIO.stack(dim = ['latitude', 'longitude'])
print(bio_stack.values)
print(bio_stack.shape)
print(bio_stack.dim.values)

Now we create a DataFrame with the environmental variables. This needs to be transposed to have the same format as the DataFrame used in training the model

In [51]:
curr_env_df = pd.DataFrame(bio_stack.values.transpose(),
                columns = var_names)
curr_env_df

We can now use this to predict suitability. The only problem here is that there are number of rows that have missing values (NaNs) representing ocean grid cells. If we try to use these to predict, the method will fail, so we need a way to ignore them. We could simply drop them from the DataFrame, but this makes it difficult to map out the predictions, as these no longer line up with the coordinates. Instead, we'll set all missing values to 0 with the `fillna()` method. This means that the model will predict for these grid cells, but we can use the mask we created when visualizing the results. 

We'll go ahead and predict suitability using the tuned random forest model (feel free to swap this out for the decision tree of xgboost model):

In [52]:
x = pe_rf_tuned.predict_proba(curr_env_df.fillna(0))
x.shape

Once this is done, we can convert it back to an xarray using the values (`x`) and the grid coordinates (which are extracted from the original `curr_env` object):

In [53]:
curr_pred = xr.DataArray(x[:,1].reshape(480,720), 
                         coords=[curr_env.coords['latitude'], 
                                 curr_env.coords['longitude']])

And now we can plot out the results:

In [54]:
curr_pred.where(mask_array == 1).plot()
plt.title("Pinus edulis\npredicted current distribution")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.show()

This shows that model captures the current distribution well, but also predicts a large area of suitability in the north-west. We can also plot the predicted presence/absence as a binary outcome. To do this, we first need to get the optimal threshold for discriminating between absence (0) and presence (1):

In [55]:
y_test_pred = pe_rf_tuned.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_test_pred[:,1])
threshold_rf = thresholds[np.argmax(tpr - fpr)]
print(threshold_rf)

And now we can plot this:

In [56]:
curr_pa = curr_pred > threshold_rf

curr_pa.where(mask_array == 1).plot()
plt.title("Pinus edulis\npredicted current distribution")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.show()

### Future climate

The previous maps are based on current (or at least end of last century) estimates of climate. We can equally predict for other time periods when we have data available. Earlier in the lab we read in climate data for the end of the century under a high emissions scenario (RCP8.5). We can go through the same steps to produce a map showing projected suitability for this species

In [57]:
bio_stack = future_env.BIO.stack(dim = ['latitude', 'longitude'])
future_env_df = pd.DataFrame(bio_stack.values.transpose(),
                columns = var_names)
future_crds = pd.DataFrame(bio_stack.dim.values)

In [58]:
x = pe_rf_tuned.predict_proba(future_env_df.fillna(0))
future_pred = xr.DataArray(x[:,1].reshape(480,720), 
                         coords=[future_env.coords['latitude'], 
                                 future_env.coords['longitude']])

In [59]:
future_pred.where(mask_array == 1).plot()
plt.title("Pinus edulis\npredicted future distribution")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.show()

And again, we can use the threshold to identify areas of suitability. 

In [60]:
future_pa = future_pred > threshold_rf

future_pa.where(mask_array == 1).plot()
plt.title("Pinus edulis\npredicted future distribution")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.show()

As a final result, we can use the two binary maps to more easily visualize how the distribution of suitable areas is projected to change. By subtracting the current binary distribution from the future binary distribution, we end up with a map with three values:

- 1: new areas of suitability
- 0: No change
- -1: loss of suitability

In [61]:
change_pa = future_pa.astype('int') - curr_pa.astype('int')
change_pa.where(mask_array == 1).plot()
plt.title("Pinus edulis\npredicted change in distribution")
plt.xlabel("longitude")
plt.ylabel("latitude")
plt.show()

## Exercise

In a previous lab, you used data from the *Sonar.csv* file to model types of object (rocks 'R' or mines 'M') using the values of a set of frequency bands. The exercise for this lab is to use one of the ensemble methods (random forest *or* boosted regression trees) to produce a new model of these data. You should use the **scikit-learn** framework to set up and test your model, and you need to do the following:

- Run the model with the default hyperparameter settings, and report the AUC for the test set [1]
- Use `GridSearchCV()` to tune the model [2]
- Report the AUC score for the tuned model, as well as the tuned parameters [1]
- Produce a variable importance plot [1]

In addition, your answer should include your code, either as a word document or jupyter notebook

## Appendix 1: Bioclimate variables

- BIO1 = Annual Mean Temperature
- BIO2 = Mean Diurnal Range (Mean of monthly (max temp - min temp))
- BIO3 = Isothermality (BIO2/BIO7) (* 100)
- BIO4 = Temperature Seasonality (standard deviation *100)
- BIO5 = Max Temperature of Warmest Month
- BIO6 = Min Temperature of Coldest Month
- BIO7 = Temperature Annual Range (BIO5-BIO6)
- BIO8 = Mean Temperature of Wettest Quarter
- BIO9 = Mean Temperature of Driest Quarter
- BIO10 = Mean Temperature of Warmest Quarter
- BIO11 = Mean Temperature of Coldest Quarter
- BIO12 = Annual Precipitation
- BIO13 = Precipitation of Wettest Month
- BIO14 = Precipitation of Driest Month
- BIO15 = Precipitation Seasonality (Coefficient of Variation)
- BIO16 = Precipitation of Wettest Quarter
- BIO17 = Precipitation of Driest Quarter
- BIO18 = Precipitation of Warmest Quarter
- BIO19 = Precipitation of Coldest Quarter